In [ ]:
# Proyecto de trayectorias aéreas

"""
El objetivo es minimizar el número de interacciones entre aviones, para ello hay que
modificar las rutas de cada avión de acuerdo a las restricciones.

La idea es considerar el espacio 3D , con 2D espaciales y una temporal (o en volá 4D)
x en [x_ini, x_fin], para fines de simplificar x en [0, X] (i.e x_ini = 0, x_fin = X)
same shit with the others

Este espacio se discretiza formando una grilla terrible cuatica, luego cada punto
de cada Trayectoria (vuelo), se ubica en una casilla, si dos puntos estan cerca (i.e. estan 
en casillas adjuntas) se calcula su distancia y se ve si hay interacción. 
El grafo a usar en cuestión es tal que cada vertice es una distribución de las N trayectorias,
dos vertices son vecinos si difieren en exactamente 1 elemento de (x,y) (t) (o tambien (z))

"""

In [ ]:
import numpy as np
import random

# Sea N el numero de trayectorias

trayectorias_originales = None 
N = len(trayectorias_originales)

""" Si es que entendí el coso, Se supone que hay que tambien hay que conocer los w_i (los (x,y) permitidos para modificar), 
los delta_i (tiempo inicial permitidos creo) y l_i (same shit with z), este ultimo no  se considera si se deja fuera el eje Z,
por otro lado, hay que cachar como definir las otras volas"""

P_w = P_l = P_d = 1 / 3

X = 
Y =
T = 
# Z =

d_x =
d_y =
d_t = 
# d_z

N_x = X / d_x #  tkm por favor que sea un entero
N_y = Y / d_y
N_t = T / d_t
# N_z

# N_v =
N_h =
t_eps = d_t / 5 



In [ ]:
# Interacciones a la mala

def C(P, Q):
    x = P[0] - Q[0]
    x = P[1] - Q[1]
    d_h = np.linalg.norm((x, y))
    if np.abs(P[3] - Q[3]) < 2*t_eps and d_h < N_h:
        return 1
    else:
        return 0
    
def interaccion(tray, i, k):
    suma = 0
    for j in range(len(tray)):
        if j!=i:
            for l in len(tray[j]):
                suma += C(tray[i][k], tray[j][l])
                
    return suma

def Interacciones(trayectorias):
    phi = 0
    for i in range(N):
        for k in range(len(trayectorias[i])):
            phi += interaccion(trayectorias, i, k)
            
    return phi
  

In [25]:
# Lo anterior es poquito ineficiente. Se da la idea de hacerlo por grillas
# No se si será la manera mas eficiente de implementarlo tho

def interaccion_por_grilla(tray):
    cas_x = []
    cas_y = []
    cas_t = []
    for x in range(N_x):
        cas_x.append([])
    for y in range(N_y):
        cas_y.append([])
    for t in range(N_t):
        cas_t.append([])
        
    phi = 0
    
    for t in tray:
        for P in i:
            n_x = int(P[0] / d_x)
            n_y = int(P[1] / d_y)
            n_t = int(P[2] / d_t)
            
            #veo si en las casillas adjuntas hay algun punto cerca
            for i in range(-1, 2):
                for j in range(-1, 2):
                    for k in range(-1, 2):
                        if len(cas_x[n_x + i])>0 and len(cas_y[n_y + j])>0 and len(cas_t[n_t + k])>0:
                            for Q in cas_x[n_x + i]:
                                if Q in cas_y[n_y + j] and Q in cas_t[n_t + k]:
                                    phi += C(P,Q)
                        
            cas_x[n_x].append(P)
            cas_y[n_y].append(P)
            cas_t[n_t].append(P)
    
    return phi
            

In [27]:
def neighborhood(trayectorias):
    t = trayectorias.copy()
    i = randint(0, len(trayectorias))
    new_t = t[i]
    u = np.random.uniform(0, 1)
    
    # para configurar la nueva trayectoria se necesita saber como las vamos a definir 
    if u < P_w:
        """cambiar w"""
    elif u < P_w + P_l:
        """cambiar l"""
    else:
        """cambiar delta"""
        
    t[i] = new_t
    
    return t

In [26]:
# como referencia: Algoritmo traido directamente del lab 3 (mas abajo esta la modificación)

def CM(t, func_beta, D_matriz, N):
    """
    Simula la cadena hasta tiempo t, se toma condicion inicial sigma(i)=i
    func_beta es la función que da el valor de beta
    según cada iteración
    """
    X = []
    X.append(np.arange(N)) 
    u = np.random.uniform(0,1,t)
    
    for n in range(t):
        beta = func_beta(n)
        
        #construyo un Y aleatorio que sea vecino de X=X_n
        d = sorted(np.arange(N))
        c = random.sample(d,2)
        a = c[0]
        b = c[1]
        sigma = np.zeros(N)
        
        for j in range(N):
            sigma[j] = X[n][j]
            
        y_1 = sigma[a]
        y_2 = sigma[b]
        sigma[b] = y_1
        sigma[a] = y_2
        
        
        cte_n = np.min((1, np.exp(-beta * (w(sigma, D_matriz, N) - w(X[n], D_matriz, N)))))
        
        if u[n] < cte_n:
            X.append(sigma)
        else:
            X.append(X[n])
            
    return X

In [ ]:
# en nuestro caso w sería el phi (interacciones), y la funcion recibiría las trayectorias iniciales

def CM(t, func_beta, trayectorias):
    """
    Simula la cadena hasta tiempo t
    func_beta es la función que da el valor de beta
    según cada iteración
    """
    X = []
    X.append(trayectorias) 
    u = np.random.uniform(0,1,t)
    
    for n in range(t):
        beta = func_beta(n)
        Y = neighborhood(X[n])
        cte_n = np.min((1, np.exp(-beta * (interaccion_por_grilla(Y) - interaccion_por_grilla(X[n])))))
        
        if u[n] < cte_n:
            X.append(Y)
        else:
            X.append(X[n])
            
    return X